## User Car Price Prediction

### 1. Problem Statement
- This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
- If user can predict the price of the car based on input features.
- Prediction results can be used to give new seller the price suggestion based on market condition.

### 2. Data Collection
- The Dataset is collected from scrapping from cardheko webiste
- The data consists of 13 column and 15411 rows.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('Data/cardekho_imputated.csv')
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [3]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [4]:
## Remove Unnecessary Columns
df.drop(columns=['car_name','brand'],axis=1,inplace=True)

In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
X = df.drop(['selling_price'],axis=1)
y = df['selling_price']

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [9]:
# Create column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
onehotcolumns = ['seller_type','fuel_type','transmission_type']



from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehotcolumns),
        ("StandardScaler",numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [10]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('OneHotEncoder', OneHotEncoder(drop='first'),
                                 ['seller_type', 'fuel_type',
                                  'transmission_type']),
                                ('StandardScaler', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [12]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [14]:
# Creating function for Evaluating the Model
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2score = r2_score(true,predicted)
    return mae,mse,r2score,rmse

In [15]:
## Begenning the model

models = {
    'XGBoost Regressor' : XGBRegressor(),
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "k-Nearest Neighbour Regressor": KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest" : RandomForestRegressor(),
    "Adaboost Regressor": AdaBoostRegressor(),
    "GradientBoost Regressor": GradientBoostingRegressor()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    ## Make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae,model_train_mse,model_train_r2score,model_train_rmse = evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_r2score,model_test_rmse = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("Model Performance for Training set")
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_train_rmse))


    print('------------------------------------------------------------------')

    print("Model Performance for Test set")
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("-  R2 Score: {:.4f}".format(model_test_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("\n")


XGBoost Regressor
Model Performance for Training set
- Mean Absolute Error: 58826.0756
- Mean Squared Error: 6745111788.4731
- R2 Score: 0.9919
- Root-Mean Squared Error: 82128.6295
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 100866.8600
- Mean Squared Error: 73095555235.8518
-  R2 Score: 0.9010
- Root-Mean Squared Error: 270361.8968


Linear Regression
Model Performance for Training set
- Mean Absolute Error: 269089.2604
- Mean Squared Error: 317614523214.2133
- R2 Score: 0.6170
- Root-Mean Squared Error: 563572.9972
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 281003.4130
- Mean Squared Error: 253401740454.3792
-  R2 Score: 0.6569
- Root-Mean Squared Error: 503390.2467


Lasso
Model Performance for Training set
- Mean Absolute Error: 269088.2206
- Mean Squared Error: 317614534310.5324
- R2 Score: 0.6170
- Root-Mean Squared Error: 563

In [16]:
ada_params = {
    'n_estimators':[50,60,70,80],
    'loss' : ['linear', 'square', 'exponential']
}

xgboost_params = {
    'learning_rate' : [0.1,0.01],
    'max_depth':[5,8,12,20,30],
    'n_estimators':[100,200,300],
    'colsample_bytree':[0.5,0.8,1,0.3,0.4]
}

In [17]:
randomcv_models = [
    ("Adaboost",AdaBoostRegressor(),ada_params),
    ("XGBoost Regressor",XGBRegressor(),xgboost_params)
]

In [18]:
from sklearn.model_selection import RandomizedSearchCV

model_params = {}

for name,model,params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=params,
                                n_iter=100,
                                cv=3,
                                verbose=2,
                                n_jobs=-1)
    random.fit(X_train,y_train)
    model_params[name] = random.best_params_

for model_name in model_params:
    print(f"------------------- Best Params for {model_name} --------------")
    print(model_params[model_name])
                                

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
------------------- Best Params for Adaboost --------------
{'n_estimators': 50, 'loss': 'linear'}
------------------- Best Params for XGBoost Regressor --------------
{'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.5}


In [20]:
## Begenning the model

models = {
    "Adaboost Regressor": AdaBoostRegressor(n_estimators=50,loss='linear'),
    "XGboost Regressor" : XGBRegressor(n_estimators=300,max_depth=5,learning_rate=0.1,colsample_bytree=0.5),
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    ## Make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae,model_train_mse,model_train_r2score,model_train_rmse = evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_r2score,model_test_rmse = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("Model Performance for Training set")
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_train_rmse))


    print('------------------------------------------------------------------')

    print("Model Performance for Test set")
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("-  R2 Score: {:.4f}".format(model_test_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("\n")


Adaboost Regressor
Model Performance for Training set
- Mean Absolute Error: 316277.4561
- Mean Squared Error: 180599082280.9103
- R2 Score: 0.7822
- Root-Mean Squared Error: 424969.5075
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 330046.2137
- Mean Squared Error: 215521980517.6805
-  R2 Score: 0.7082
- Root-Mean Squared Error: 464243.4496


XGboost Regressor
Model Performance for Training set
- Mean Absolute Error: 72734.1455
- Mean Squared Error: 11866586010.7507
- R2 Score: 0.9857
- Root-Mean Squared Error: 108933.8607
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 104239.7541
- Mean Squared Error: 88103850315.9769
-  R2 Score: 0.8807
- Root-Mean Squared Error: 296822.9275


